In [34]:
import os
import requests

In [ ]:
# replace after you deploy your llm (deploy_llm.py)
LLM_HOST = ""
LLM_PORT = ""

In [35]:
prompt = """
Тебе нужно проанализировать документ и извлечь следующие переменные в виде JSON ответа

Документ представляет собой договор между поставщиком и заказчиком на какую либо закупку

Извлеки только те переменные которые находятся в тексте и выведи в виде JSON объекта без markdown

Нужен только JSON, больше ничего писать не надо

Проверь, что результирующий JSON корректен ДОБавь закрывающую фигурную скобку, если её нет

Если какой-либо переменной не нашлось, то не добавляй ключ в JSON

Старайся приводить значения существующих переменных к именительному падежу

Переменные для реквизитов:
contract_date - дата контракта [ 26.04.2024 ]
contract_number - номер контракта [ 62143 ]
customer_name - название заказчика [ Муниципальное бюджетное общеобразовательное учреждение «Средняя общеобразовательная школа №5» муниципального образования город Ноябрьск ]
customer_agent - должность представителя заказчика [ директора ]
customer_agent_name - ФИО представителя заказчика [ Савенковой  Оксаны  Васильевны ] 
customer_act - на основании чего действует заказчик [ Приказа № 01-05-1053/24 от 20.06.2024 ]
supplier_name - название поставщика [ Общество сограниченной ответственностью "Омега-Консалтинг" ]
supplier_agent - должность представителя поставщика [ директора ]
supplier_agent_name - ФИО представителя поставщика [ Савенковой  Оксаны  Васильевны ]
supplier_act - на основании чего действует поставщик [ Приказа № 01-05-1053/24 от 20.06.2024 ]
city - город [г. Москва]
article_details_number - номер главы в которой лежат реквизиты [15]
article_details_name - название главы в которой лежат реквизиты [Адреса, реквизиты и подписи Сторон]

{document}
"""

In [ ]:
dir = "../contract/file/storage/plain_text"

metadata_list = {}
for file in os.listdir(dir):
    if "2.txt" not in file:
        continue

    with open(os.path.join(dir, file), 'r') as f:
        file_text = f.read()

    # Разбиваем текст на чанки
    chunks = [file_text[i:i + 5000] for i in range(0, len(file_text), 5000)]
    
    all_metadata = []
    for chunk in chunks:
        final_prompt = prompt.format(document=chunk)

        payload = {
            "prompt": final_prompt
        }

        response = requests.post(f"http://{LLM_HOST}:{LLM_PORT}/request", json=payload)
        
        # Предполагая, что в результате может быть несколько метаданных, добавляем их все в один список
        all_metadata.append(response.json().get('result', {}))

    metadata_list[file] = all_metadata

In [ ]:
metadata_list.items()

In [ ]:
import json


for name, metadata in metadata_list.items():
    
    try:
        json_dict = json.loads(metadata.replace("```json", "").replace("```", ""))
        with open(os.path.join(dir, name.replace(".txt", '.json')), 'w') as f:
            json.dump(json_dict, f, ensure_ascii=False, indent=4)
    except:
        print(metadata)